In [1]:
import carla 
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math

In [2]:
import sys
sys.path.append('/home/carla-simulator/PythonAPI/carla') #Adds the directory to the path so that modules
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [3]:
# define speed contstants
PREFERRED_SPEED = 90 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

In [4]:
client = carla.Client('localhost', 2000)

client.load_world('Town01')

world = client.get_world()

spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*mini')
start_point = spawn_points[2]

vehicle = world.try_spawn_actor(vehicle_bp[1], start_point)

In [5]:
#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1

In [6]:
#camera mount offset on the car - you can tweak these to have the car in view or not
CAMERA_POS_Z = 3 
CAMERA_POS_X = -5 

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') 
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

In [7]:
def maintain_speed(s):#(input=current speed),(output=accelerator input to the car)
    
    if s >= PREFERRED_SPEED: ##high speed## s>30
        return 0 #7nfdl zy me7naa (msh 7zwd el acceleration)
    
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD: ##low speed## s<(20-2) = s<18
        return 0.8 # 7zwd el sor3a (7ados 3l bnzen bnsba 0.8) % of "full gas"
    
    else: # 28<s<30
        return 0.4 #7dzwd 7aga 5afefa (0.4% of full gas) 

In [8]:
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))


In [9]:

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])


In [10]:
#main loop 
quit = False

while True:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break
    image = camera_data['image']
    
    steering_angle = 0 # we do not have it yet
    # to get speed we need to use 'get velocity' function
    v = vehicle.get_velocity()
    # if velocity is a vector in 3d
    # then speed is like hypothenuse in a right triangle
    # and 3.6 is a conversion factor from meters per second to kmh
    # e.g. kmh is 1000 meters and one hour is 60 min with 60 sec = 3600 sec
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    # now we add the speed to the window showing a camera mounted on the car
    image = cv2.putText(image, 'Speed: '+str(int(speed))+' kmh', org, 
                        font, fontScale, color, thickness, cv2.LINE_AA)
    # this is where we used the function above to determine accelerator input
    # from current speed
    estimated_throttle = maintain_speed(speed)
    # now we apply accelerator
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, 
                                               steer=steering_angle))
    cv2.imshow('RGB Camera',image)

#clean up
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [11]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

In [12]:
client = carla.Client('localhost', 2000)

client.load_world('Town01')

world = client.get_world()

spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*mini')
start_point = spawn_points[3]

start_point2= carla.Transform(carla.Location(x=335.489990, y=273.743317, z=0.043043), carla.Rotation(pitch=0.000000, yaw=90.000046, roll=0.000000))

vehicle = world.try_spawn_actor(vehicle_bp[1], start_point)

In [13]:
# get the car's position on the map 
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=299.399994, y=59.330032, z=0.299888), Rotation(pitch=0.000000, yaw=0.000000, roll=0.000000))


In [14]:
#3shan nela2y el car badl mndwr bel mouse wel wasd: (location above(z=2.5) and behind(x=-4) the car)
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [15]:
# initial spawn point is the same - just 0.6m higher off the ground
print(start_point)

Transform(Location(x=299.399994, y=59.330036, z=0.300000), Rotation(pitch=0.000000, yaw=-0.000092, roll=0.000000))


In [16]:
#send vehicle off
vehicle.set_autopilot(True)

In [17]:
# get actual position from the car moving
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=299.399994, y=59.330032, z=0.003707), Rotation(pitch=0.000000, yaw=0.000000, roll=0.000000))


In [18]:
# now look at the map
town_map = world.get_map()

In [19]:
type(town_map)

carla.libcarla.Map

In [20]:
roads = town_map.get_topology()

In [21]:
#have a look at a waypoint - it is transform wrapped as a waypoint
print(roads[0][0])

Waypoint(Transform(Location(x=144.990234, y=59.493351, z=0.000000), Rotation(pitch=0.000000, yaw=-0.006982, roll=0.000000)))


In [22]:
len(roads)

160

In [23]:
import sys
sys.path.append('/home/carla-simulator/PythonAPI/carla') #Adds the directory to the path so that modules
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [24]:
sampling_resolution = 2

grp = GlobalRoutePlanner(town_map, sampling_resolution)

point_a = carla.Location(x=299.399994, y=129.750000, z=0.299536)
point_b = carla.Location(x=107.175446, y=129.416809, z=0.003263)

route = grp.trace_route(point_a, point_b)

for waypoints in route:
    world.debug.draw_string(waypoints[0].transform.location, '^', draw_shadow=False,
                            color = carla.Color(r=0, g=0, b=255), life_time=1000.0,
                            persistent_lines=True)



In [25]:
# utility script of destruction

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [26]:
# now we define 2 cars
truck_bp = world.get_blueprint_library().filter('*firetruck*')
mini_bp = world.get_blueprint_library().filter('*cooper_s*')

#start first car in alredy defined start point
truck = world.try_spawn_actor(truck_bp[0], start_point)

In [27]:
# tweak spectator position to watch the show

spectator = world.get_spectator()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]

spectator_pos = carla.Transform(start_point.location + carla.Location(x=20,y=10,z=4),
                                carla.Rotation(yaw = start_point.rotation.yaw -155))

spectator.set_transform(spectator_pos)

In [28]:
print(len(spawn_points))

255


In [29]:
transform = carla.Transform(carla.Location(x=200, y=190, z=40), carla.Rotation(yaw=180))

In [30]:
# drop the Mini the sky - watch what happens after

#spawn it first somewhere else
mini = world.try_spawn_actor(mini_bp[1], transform)

mini_pos = carla.Transform(start_point.location + carla.Location(x=-10,z=10),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0))
mini.set_transform(mini_pos)